<a href="https://colab.research.google.com/github/niklasforestman/RC-AI-Jetson/blob/master/CarSoftware3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Selbstfahrendes RC-Fahrzeug 
In diesem Notebook wird der Code präsentiert, dem es unserem RC-Fahrzeug ermöglicht Bilder aufzunehmen, mit denen eine neuronales Netz trainiert wird, welches anschließend unser Fahrzeug steuern soll.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import keras
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import cv2
import pandas as pd
import ntpath
import random


### Task 1: Trainingsbilder mit Lenkwinkel aufnehmen


In [ ]:
while True:
  img = Kamerabild
  save_img(path, name, img)
  write_to_csv img_name, Controller_Value_Lenkwinkel, Controller_Value_Längs


Hello World


### Task 2: Trainingsbilder augmentieren

In [ ]:
# Ein Bild wird gezoomt
def zoom(image):
  zoom = iaa.Affine(scale=(1, 1.3)) #Zoom mit scale, Affine sorgt dafür, dass Linien im Bild erhalten bleiben
  image = zoom.augment_image(image) #Apply die Zoom-Parameter auf das Bild
  return image 

In [ ]:
# Verschieben des Bildes
def pan(image): 
  pan = iaa.Affine(translate_percent= {"x" : (-0.1, 0.1), "y": (-0.1, 0.1)})
  image = pan.augment_image(image)
  return image

In [ ]:
# Helligkeit eines Bildes verändern
def img_random_brightness(image):
    brightness = iaa.Multiply((0.2, 1.2))
    image = brightness.augment_image(image)
    return image

In [ ]:
# Ein Bild wird gespiegelt
def img_random_flip(image, steering_angle):
    image = cv2.flip(image,1)
    steering_angle = -steering_angle
    return image, steering_angle

Die Funktion random_augment() führt auf ein übergebenes Bild die möglichen Augmentations aus

In [ ]:
def random_augment(image, steering_angle):
    image = mpimg.imread(image)
    if np.random.rand() < 0.5:
      image = pan(image)
    if np.random.rand() < 0.5:
      image = zoom(image)
    if np.random.rand() < 0.5:
      image = img_random_brightness(image)
    if np.random.rand() < 0.5:
      image, steering_angle = img_random_flip(image, steering_angle)
    
    return image, steering_angle

Die Funktion img_preprocess() nimmt ein Bild und verarbeitet es für das Modell

In [ ]:
def img_preprocess(img):
    #img = img[60:135,:,:] #Das Bild kann zugeschnitten werden auf einen bestimmten Bereich
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV) #colormap für NVIDIA-Model
    #img = cv2.GaussianBlur(img,  (3, 3), 0) #Blur um Bild zu smoothen
    img = cv2.resize(img, (200, 66)) #Auf richtige Größe für Model bringen
    img = img/255 #Bild normalisieren
    return img


### Task 3: Neuronales Netz aufbauen

Die Funktion batch_generator() generiert für das Training des Modells einen Batch an Bildern und zugehörigem Lenkwinkel (muss noch um Gas/Bremse erweitert werden)

In [ ]:
def batch_generator(image_paths, steering_ang, batch_size, istraining):
  
  while True:
    batch_img = []
    batch_steering = []
    
    for i in range(batch_size):
      random_index = random.randint(0, len(image_paths) - 1)
      
      if istraining: #falls die Funktion im Modelltraining aufgerufen werden soll, dann sollen die Bilder augmentiert werden
        im, steering = random_augment(image_paths[random_index], steering_ang[random_index])
     
      else:
        im = mpimg.imread(image_paths[random_index])
        steering = steering_ang[random_index]
      
      im = img_preprocess(im) 
      batch_img.append(im)
      batch_steering.append(steering)
    yield (np.asarray(batch_img), np.asarray(batch_steering))

Hier wird das Convolutional Neural Network definiert

In [ ]:
def nvidia_model():
  model = Sequential()
  model.add(Conv2D(24, kernel_size=(5,5), strides=(2,2), input_shape=(66,200,3),activation='relu'))
  
  model.add(Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'))
  model.add(Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'))
  model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
  model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
  model.add(Dropout(0.5))
  
  
  model.add(Flatten())
  
  model.add(Dense(100, activation = 'elu')) #elu ist kein Schreibfehler
#   model.add(Dropout(0.5))
  
  model.add(Dense(50, activation = 'elu')) #elu verhindert ein dead-relu
#   model.add(Dropout(0.5))
  
  model.add(Dense(10, activation = 'elu'))
#   model.add(Dropout(0.5))
 
  model.add(Dense(1))
  
  optimizer = Adam(lr=1e-3)
  model.compile(loss='mse', optimizer=optimizer) #Mean Squared Error
  return model

model = nvidia_model()
print(model.summary())
history = model.fit_generator(batch_generator(X_train, y_train, 100, 1),
                                  steps_per_epoch=300, 
                                  epochs=10,
                                  validation_data=batch_generator(X_valid, y_valid, 100, 0),
                                  validation_steps=200,
                                  verbose=1,
                                  shuffle = 1)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training', 'validation'])
plt.title('Loss')
plt.xlabel('Epoch')

model.save('model.h5') #Hier kann das Modell gespeichert werden, um es später zu verwenden

### Task 4: Anwendung des neuronalen Netzes im Fahrzeug